In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegressionCV
from sklearn.svm import SVC
%matplotlib inline
import src.utils

In [2]:
df = src.utils.gene_variation_dataframe()

In [3]:
from xgboost import XGBClassifier

In [4]:
xgb = XGBClassifier(n_estimators=1000,learning_rate=.01,
                    objective='multi:softprob',num_classes=9,
                   subsample=.5,colsample_bytree=.5,max_depth=3,
                   n_jobs=-1)
from sklearn.model_selection import cross_val_score
from sklearn.utils import shuffle

In [6]:
df_train = pd.read_csv('../data/training_variants.csv')
X_train = df[:df_train.shape[0]]

x,y = shuffle(X_train,df_train.Class)
cvs = -cross_val_score(xgb, x, y, scoring='neg_log_loss',n_jobs=-1,cv=5).mean()
print('xgb : {:.3f}'.format(cvs))

xgb : 0.925


In [ ]:
for lr in (.001,.005,.01,.05):
    xgb = XGBClassifier(n_estimators=5000,learning_rate=lr,
                    objective='multi:softprob',num_classes=9,
                   subsample=.5,colsample_bytree=.5,max_depth=3,
                       n_jobs=-1)
    cvs = -cross_val_score(xgb, x, y, scoring='neg_log_loss',n_jobs=-1,cv=5)
    print('xgb (lr={:.2f}) : {:.3f} +/- {:.3f}'.format(lr,cvs.mean(),cvs.std()))

In [ ]:
for depth in (2,3,4,5):
    xgb = XGBClassifier(n_estimators=5000,learning_rate=lr,
                    objective='multi:softprob',num_classes=9,
                   subsample=.5,colsample_bytree=.5,max_depth=depth,
                       n_jobs=-1)
    cvs = -cross_val_score(xgb, x, y, scoring='neg_log_loss',n_jobs=-1,cv=5)
    print('xgb (lr={:.2f}) : {:.3f} +/- {:.3f}'.format(lr,cvs.mean(),cvs.std()))